In [1]:
!pip install beautifulsoup4



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install selenium



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install webdriver_manager

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:

# # 크롬 드라이버 생성
# driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# # Google 웹사이트에 접속
# driver.get("https://www.google.com")

# # 검색 입력창 찾기 (검색창의 이름이 'q')
# search_box = driver.find_element(By.NAME, 'q')

# # 검색어 입력
# search_box.send_keys('Jennie')

# # 검색 실행
# search_box.submit()
# time.sleep(5)

# # 결과 페이지 스크린샷 저장
# driver.save_screenshot('search_results.png')

# # 브라우저 종료
# driver.quit()

In [20]:
import requests
import time
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd


# 크롬 드라이버 생성
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))


# 페이지 열기
url = "https://unipass.customs.go.kr/clip/index.do"
driver.get(url)

# JavaScript가 실행될 시간을 기다리기
driver.implicitly_wait(10)  # 최대 10초 대기

# 페이지의 HTML 가져오기
# soup = BeautifulSoup(driver.page_source, 'html.parser')

# 세계HS 창을 찾아 click_box 변수에 저장 (By.CLASS_NAME 방식)
click_box = driver.find_element(By.CLASS_NAME,"M_ULS0200000000")
click_box.click()

# HS해설서 창을 찾아 click_box 변수에 저장 (By.CLASS_NAME 방식)
click_box = driver.find_element(By.XPATH, '//*[@id="LEFTMENU_LNK_UI-ULS-0202-001Q"]/span')
click_box.click()

# 조회 창을 찾아 click_box 변수에 저장 (By.XPATH 방식)
click_box = driver.find_element(By.XPATH, '//*[@id="searchVo"]/div[2]/footer/button/span')
driver.execute_script("arguments[0].click();", click_box)


# Wait for the table to load
time.sleep(5)  # Adjust the wait time as needed

# Locate the table
table = driver.find_element(By.XPATH, '//*[@id="ULS0202001Q_T1_table1"]/tbody')

# Parse the table HTML using BeautifulSoup
soup = BeautifulSoup(table.get_attribute("outerHTML"), 'html.parser')

# Extract table rows
rows = soup.find_all('tr')

# Initialize an empty DataFrame
data = pd.DataFrame(columns=["구분", "년도", "단위", "HS분류", "내용", "Left Area", "Right Area"])

# # Iterate through the rows and extract data
# for row in rows:
#     cols = row.find_all('td')
#     cols = [col.text.strip() for col in cols]  # Extract text and clean up whitespace
#     if cols:  # Check if the row has data
#         # Click on the `dtlInfo` link
#         dtl_link = row.find('a', class_='dtlInfo')
#         if dtl_link:
#             dtl_element = driver.find_element(By.LINK_TEXT, dtl_link.text)
#             dtl_element.click()
#             time.sleep(2)  # Wait for the leftArea and rightArea to load

#             # Extract text from leftArea
#             left_area = driver.find_element(By.ID, 'divLft_tab4')
#             left_area_text = left_area.text.strip()

#             # Extract text from rightArea
#             right_area = driver.find_element(By.ID, 'divRght_tab4')
#             right_area_text = right_area.text.strip()

#             # Go back to the main page to continue scraping
#             # driver.back()
#             time.sleep(2)  # Wait for the page to load

#             # Add data to the DataFrame
#             data.loc[len(data)] = cols + [left_area_text, right_area_text]

# Loop through all 135 pages
for page in range(1, 136):
    print(f"Processing page {page}...")
    
    # Locate the table
    table = driver.find_element(By.XPATH, '//*[@id="ULS0202001Q_T1_table1"]/tbody')
    
    # Parse the table HTML using BeautifulSoup
    soup = BeautifulSoup(table.get_attribute("outerHTML"), 'html.parser')
    
    # Extract table rows
    rows = soup.find_all('tr')
    
    # Iterate through the rows and extract data
    for row in rows:
        cols = row.find_all('td')
        cols = [col.text.strip() for col in cols]  # Extract text and clean up whitespace
        if cols:  # Check if the row has data
            # Click on the `dtlInfo` link
            dtl_link = row.find('a', class_='dtlInfo')
            if dtl_link:
                dtl_element = driver.find_element(By.LINK_TEXT, dtl_link.text)
                dtl_element.click()
                time.sleep(1)  # Wait for the leftArea and rightArea to load

                # Extract text from leftArea
                left_area = driver.find_element(By.ID, 'divLft_tab4')
                left_area_text = left_area.text.strip()

                # Extract text from rightArea
                right_area = driver.find_element(By.ID, 'divRght_tab4')
                right_area_text = right_area.text.strip()

                # Go back to the main page to continue scraping
                time.sleep(1)  # Wait for the page to load

                # Add data to the DataFrame
                data.loc[len(data)] = cols + [left_area_text, right_area_text]
    
    # Navigate to the next page
    try:
        next_page = driver.find_element(By.XPATH, f'//div[@class="paging"]//a[@href="#{page + 1}"]')
        next_page.click()
        time.sleep(5)  # Wait for the page to load
    except Exception as e:
        print(f"Could not navigate to page {page + 1}: {e}")
        break

# Display the DataFrame

# Save the data to a CSV file
data.to_csv('HSdescription_data.csv', index=False, encoding='utf-8-sig')

Processing page 1...
Processing page 2...
Processing page 3...
Processing page 4...
Processing page 5...
Processing page 6...
Processing page 7...
Processing page 8...
Processing page 9...
Processing page 10...
Processing page 11...
Processing page 12...
Processing page 13...
Processing page 14...
Processing page 15...
Processing page 16...
Processing page 17...
Processing page 18...
Processing page 19...
Processing page 20...
Processing page 21...
Processing page 22...
Processing page 23...
Processing page 24...
Processing page 25...
Processing page 26...
Processing page 27...
Processing page 28...
Processing page 29...
Processing page 30...
Processing page 31...
Processing page 32...
Processing page 33...
Processing page 34...
Processing page 35...
Processing page 36...
Processing page 37...
Processing page 38...
Processing page 39...
Processing page 40...
Processing page 41...
Processing page 42...
Processing page 43...
Processing page 44...
Processing page 45...
Processing page 46.

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"link text","selector":"제70류 유리와 유리제품 주: 1. 이 류에서 다음 각 목의 것은 제외한다. 가. 제3207호의 물품[예: 법랑ㆍ유약ㆍ유리프리트(frit)와 가루 모양ㆍ알갱이 모양ㆍ플레이크(flake) 모양인 그 밖의 유리] 나. 제71류의 물품(예:  모조 신변장식용품) 다. 제8544호의 광섬유 케이블, 애자(제8546호..."}
  (Session info: chrome=131.0.6778.86); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x012333E3+25059]
	(No symbol) [0x011BCDE4]
	(No symbol) [0x0109BEC3]
	(No symbol) [0x010DFD86]
	(No symbol) [0x010DFFCB]
	(No symbol) [0x0111D952]
	(No symbol) [0x01101F44]
	(No symbol) [0x0111B51E]
	(No symbol) [0x01101C96]
	(No symbol) [0x010D3FAC]
	(No symbol) [0x010D4F3D]
	GetHandleVerifier [0x01525543+3113795]
	GetHandleVerifier [0x0153A20A+3198986]
	GetHandleVerifier [0x015329E2+3168226]
	GetHandleVerifier [0x012D3250+680016]
	(No symbol) [0x011C572D]
	(No symbol) [0x011C29D8]
	(No 